<a href="https://colab.research.google.com/github/tr-dev-bc/Modern_CV_Assignments/blob/main/step_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt         # used to plot error later
from torchvision import datasets, models, transforms


In [48]:


# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"  # switch for gpu else cpu

# Load and normalize the Fashion-MNIST dataset
data_folder = '~/FMNIST'
fmnist = datasets.FashionMNIST(data_folder, download=True, train=False)
tr_images = fmnist_train.data
tr_targets = fmnist_train.targets
val_images = fmnist.data          # getting validation images
val_targets = fmnist.targets        # getting validation labels


In [49]:
class FMNISTDataset(Dataset):
    def __init__(self, x, y):
        x = x.float() / 255  # Normalize images to [0, 1]
        x = x.view(-1, 1, 28, 28)
        self.x, self.y = x, y

    def __getitem__(self, ix):
        x, y = self.x[ix], self.y[ix]  # unpack tuple into variables
        return x.to(device), y.to(device)

    def __len__(self):
        return len(self.x)

In [50]:
def get_data():
    train = FMNISTDataset(tr_images, tr_targets)
    trn_dl = DataLoader(train, batch_size=64, shuffle=True)         # batch size is 64
    return trn_dl

# Adding validation function

In [51]:
# load validation data
def get_val_data():
    val = FMNISTDataset(val_images, val_targets)
    val_dl = DataLoader(val, batch_size=64, shuffle=False)         # batch size is 64
    return val_dl

val_loader = get_val_data()

In [52]:
def get_model():                  # Here i added back the layers from the RES model
    class neuralnet(nn.Module):
        def __init__(self):
            super().__init__()
            self.conv_layer = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1)
            self.pool = nn.MaxPool2d(kernel_size=2)
            self.flatten = nn.Flatten()
            self.input_to_hidden_layer = nn.Linear(32 * 13 * 13, 128)
            self.batch_norm = nn.BatchNorm1d(128)
            self.hidden_layer_activation = nn.ReLU()
            self.hidden_to_output_layer = nn.Linear(128, 10)

        def forward(self, x):
            x = self.conv_layer(x)
            x = self.pool(x)
            x = self.flatten(x)
            x = self.input_to_hidden_layer(x)
            x0 = self.batch_norm(x)
            x1 = self.hidden_layer_activation(x0)
            x2 = self.hidden_to_output_layer(x1)
            return x2

    model = neuralnet().to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr=1e-2)
    return model, loss_fn, optimizer

# ADDING THE CAMS

In [53]:
# Function to generate and display CAMs
def generate_cam(model, images, target_layer, class_index):
    model.eval()
    with torch.no_grad():
        # Forward pass
        output = model(images)

        # Get the class score for the specified class index
        class_score = output[0, class_index]

        # Backward pass to get gradients
        model.zero_grad()
        class_score.backward()

        # Get the gradients of the target layer
        gradients = target_layer.weight.grad  # Use weight.grad for the layer
        activations = target_layer(images).detach()

        # Compute the weights
        weights = torch.mean(gradients, dim=[0, 2, 3])  # Global Average Pooling on gradients

        # Create the CAM
        cam = torch.zeros(activations.shape[2:], dtype=torch.float32)
        for i in range(weights.shape[0]):
            cam += weights[i] * activations[0, i, :, :]

        # Normalize CAM
        cam = nn.ReLU()(cam)
        cam = cam - cam.min()
        cam = cam / cam.max()
        return cam

# Display the CAMS (4)

In [55]:

def display_cams(num_images=5):
    val_dl = get_val_data()  # Use the existing validation data loader
    images, labels = next(iter(val_dl))  # Get the images and labels

    for i in range(num_images):
        image = images[i].unsqueeze(0)  # Add batch dimension
        label = labels[i].item()
        cam = generate_cam(model, image, target_layer, label)

        # Plot the image and the CAM
        plt.subplot(1, 2, 1)
        plt.imshow(image[0].cpu().numpy(), cmap='gray')
        plt.title(f'Image: {label}')
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow(cam.cpu().numpy(), cmap='jet', alpha=0.5)  # Overlay CAM
        plt.title('Class Activation Map')
        plt.axis('off')

        plt.show()